In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os 

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator


from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [3]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')

In [4]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])

target = np.column_stack([data['theta23'], data['delta']])

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))

    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_3'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_4'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_5'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_6'))

#     model_DNN_1.add(Dense(32, activation='relu', name = 'dense_4'))
    model_DNN.add(Dense(2, activation='relu', name = 'output'))
#     model_DNN_1.add(ActivityRegularization(l2=0.1, name = 'Regularization'))
#     model_DNN.add(Dropout(0.00001))
    
    # model_opt = keras.optimizers.Adadelta()
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="mean_squared_error",#keras.losses.binary_crossentropy
                              optimizer=model_opt,
                              metrics=['mse'])

    model_DNN.summary()
    return model_DNN

In [6]:
model = DNN_Model("regression")
check_list=[]
csv_logger = CSVLogger("./DNN_training_log.csv")
# checkpoint = ModelCheckpoint(
#                     filepath= "./"+str(model)+"_KFold/DNN_"+str(model)+"_Models_"+str(int(pt_min))+str(int(pt_max))+"/" + str(model) + "_DNN_checkmodel_"+str(model_index)+".h5",
#                     save_best_only=True,
#                     verbose=0)
# earlystopping = EarlyStopping(
#                     monitor="val_loss",
#                     min_delta=0,
#                     patience=20,
#                     verbose=0,
#                     mode="auto",
#                     baseline=None,
#                     restore_best_weights=False,
#                 )

# check_list.append(checkpoint)
check_list.append(csv_logger)
# check_list.append(earlystopping)

Model: "Model_DNN_regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 144)               576       
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              148480    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_6 (Dense)              (None, 1024)     

In [7]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 74s 6ms/step - loss: 1125.8205 - mse: 1125.8205 - val_loss: 322.6804 - val_mse: 322.6804
Epoch 2/20
12516/12516 [==============================] - 72s 6ms/step - loss: 602.1207 - mse: 602.1207 - val_loss: 319.5447 - val_mse: 319.5447
Epoch 3/20
12516/12516 [==============================] - 72s 6ms/step - loss: 563.2264 - mse: 563.2264 - val_loss: 261.9347 - val_mse: 261.9347
Epoch 4/20
12516/12516 [==============================] - 71s 6ms/step - loss: 553.6387 - mse: 553.6387 - val_loss: 315.6119 - val_mse: 315.6119
Epoch 5/20
12516/12516 [==============================] - 72s 6ms/step - loss: 523.4462 - mse: 523.4462 - val_loss: 271.3745 - val_mse: 271.3745
Epoch 6/20
12516/12516 [==============================] - 72s 6ms/step - loss: 511.4111 - mse: 511.4111 - val_loss: 270.4550 - val_mse: 270.4550
Epoch 7/20
12516/12516 [==============================] - 72s 6ms/step - loss: 500.0612 - mse: 500.0612 - val_loss: 214.3922 - v

In [8]:
index = 1
while os.path.isfile('/home/ML4NO/ML_DUNE/Regression/models/0910_2params_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML_DUNE/Regression/models/0910_2params_{}.h5'.format(index))

In [ ]:
scale_steps = np.logspace(-3, 0, 100)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

3125/3125 [==============================] - 9s 3ms/step - loss: 263.8452 - mse: 263.8452
Epoch 1/5
12516/12516 [==============================] - 72s 6ms/step - loss: 443.4307 - mse: 443.4307 - val_loss: 222.6053 - val_mse: 222.6053
Epoch 2/5
12516/12516 [==============================] - 73s 6ms/step - loss: 435.7136 - mse: 435.7136 - val_loss: 187.6060 - val_mse: 187.6060
Epoch 3/5
 3626/12516 [=======>......................] - ETA: 49s - loss: 450.0087 - mse: 450.0087

In [ ]:
model_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_2params_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_2params_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(20):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_PoissonTrain/0910_2params_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))